In [233]:
import pandas as pd
import numpy as np
import datetime

#download data, organize into arrays

df = pd.read_csv("mly2275.csv", comment='#')
#print(df.head())

X1=df.iloc[:, 0]
X2=df.iloc[:, 1]

#making X an array of dates 
X=[]
for i in range(np.size(X1, 0)):
    X.append(datetime.date(X1[i], X2[i], 1))
X=np.array(X)

y1=df.iloc[:, 2]
y2=df.iloc[:, 3]
y3=df.iloc[:, 4]
y4=df.iloc[:, 5]
y5=df.iloc[:, 6]
y6=df.iloc[:, 7]
y7=df.iloc[:, 8] 
y8=df.iloc[:, 9]
y9=df.iloc[:, 10]
y10=df.iloc[:, 11]

print(df.dtypes) #some of our columns came out as 'object' (string) --> we'll have to fix that
#                --> where the data's missing, use average of 2 surrounding points It's not perfect, but should be sufficient

for i in range(np.size(y7, 0)):
    if y7[i]=='None':
        y7[i]=(float(y7[i-1])+float(y7[i+1]))/2
y7=y7.astype(float)
for i in range(np.size(y8, 0)):
    if y8[i]=='None':
        y8[i]=(float(y8[i-1])+float(y8[i+1]))/2
y8=y8.astype(float)
for i in range(np.size(y9, 0)):
    if y9[i]=='None':
        y9[i]=(float(y9[i-1])+float(y9[i+1]))/2
y9=y9.astype(float)
for i in range(np.size(y10, 0)):
    if y10[i]=='None':
        y10[i]=None
y10=y10.astype(float)

#we're not including hours of sun in our data, because it's missing for the past decade
y=np.column_stack((y1, y2, y3, y4, y5, y6, y7, y8, y9))

year       int64
month      int64
meant    float64
maxtp    float64
mintp    float64
mnmax    float64
mnmin    float64
rain     float64
gmin      object
wdsp      object
maxgt     object
sun       object
dtype: object


C:\Users\Owner\Anaconda3\lib\site-packages\ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Owner\Anaconda3\lib\site-packages\ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Owner\Anaconda3\lib\site-packages\ipykernel_launcher.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Owner\Anaconda3\lib\site-packages\ipykernel_launcher.py:47: SettingWithCopyWarning: 
A valu

In [234]:

#visualise data

import matplotlib.pyplot as plt
%matplotlib widget

#timeline - each variable on its own

f=plt.figure(figsize=(10,9))
f.add_subplot(10, 1, 1)
plt.plot(X, y1, color='orange', label='mean temp')
f.add_subplot(10, 1, 2)
plt.plot(X, y2, color='red', label='max temp')
f.add_subplot(10, 1, 3)
plt.plot(X, y3, color='blue', label='min temp')
f.add_subplot(10, 1, 4)
plt.plot(X, y4, color='red',linestyle='dashed', label='mean max temp')
f.add_subplot(10, 1, 5)
plt.plot(X, y5, color='blue',linestyle='dashed', label='mean min temp')
f.add_subplot(10, 1, 6)
plt.plot(X, y6, color='lightsteelblue', label='rain')
f.add_subplot(10, 1, 7)
plt.plot(X, y7, color='green', label='grass mean temp')
f.add_subplot(10, 1, 8)
plt.plot(X, y8, color='plum', label='wind')
f.add_subplot(10, 1, 9)
plt.plot(X, y9, color='purple', label='highest gust')
f.add_subplot(10, 1, 10)
plt.plot(X, y10, color='gold', label='sun') #note that sun data disappears around 2010 - 2015
f.legend()
plt.show()

#last 30 yrs - comparing variables that seem related --> slice [612:None]
f=plt.figure(figsize=(10,9))
f.add_subplot(4, 1, 1)
plt.plot(X[612:None], y1[612:None], color='orange', label='mean temp')
plt.plot(X[612:None], y2[612:None], color='red', label='max temp')
plt.plot(X[612:None], y3[612:None], color='blue', label='min temp')
plt.legend()
f.add_subplot(4, 1, 2)
plt.plot(X[612:None], y4[612:None], color='red', label='mean max temp')
plt.plot(X[612:None], y5[612:None], color='blue', label='mean min temp')
plt.legend()
f.add_subplot(4, 1, 3)
plt.plot(X[612:None], y6[612:None], color='lightsteelblue', label='rain')
plt.plot(X[612:None], y10[612:None], color='gold', label='sun') #note that sun data disappears around 2010 - 2015
plt.legend()
f.add_subplot(4, 1, 4)
plt.plot(X[612:None], y8[612:None], color='plum', label='wind')
plt.plot(X[612:None], y9[612:None], color='purple', label='highest gust')
plt.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [235]:
#prep data for model
from sklearn.linear_model import Ridge
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures

#transform X datetime into a numerical value for the model
X1=[]; X2=[]
for z in range(np.size(X)):
    X1.append(X[z].year)
    X2.append(X[z].month)
X1=np.array(X1);X2=np.array(X2)
X=np.column_stack((X1, X2))
poly = PolynomialFeatures(2)
X = poly.fit_transform(X)

In [247]:
#Ridge Regression
C_range=[1, 5, 10, 20, 50, 75, 100, 150, 200]
MSE=[]
std_error=[]
for C in C_range:
    ridge= Ridge(alpha=1/(2*C))
    kf = KFold(n_splits=5)
    temp=[]
    for train, test in kf.split(X):
        ridge.fit(X[train], y[train])
        ypred = ridge.predict(X[test])
        temp.append(mean_squared_error(y[test],ypred))
    temp=np.array(temp)
    MSE.append(temp.mean())
    std_error.append(temp.std())
MSE=np.array(MSE)
std_error=np.array(std_error)
fig = plt.figure()
plt.errorbar(C_range, MSE, yerr=std_error, linewidth=3)
plt.xlabel('C')
plt.ylabel('Mean square error')
plt.show()
plt.figure()
plt.plot(X[test][:,2], ypred[:,2], label='preds')
plt.plot(X[test][:,2], y[test][:,2], label='actual values')
plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [246]:
#test on tiny y2021 set
print(X)
X2021=[[2021, 1], [2021, 2], [2021, 3], [2021, 4], [2021, 5], [2021, 6],
       [2021, 7], [2021, 8], [2021, 9], [2021, 10], [2021, 11], [2021, 12]]
X2021= poly.fit_transform(X2021)
preds=ridge.predict(X2021)
plt.figure()
plt.plot(X2021[:,2], preds[:,2])
plt.show()
print(X2021)

[[1.000000e+00 1.939000e+03 1.000000e+01 3.759721e+06 1.939000e+04
  1.000000e+02]
 [1.000000e+00 1.939000e+03 1.100000e+01 3.759721e+06 2.132900e+04
  1.210000e+02]
 [1.000000e+00 1.939000e+03 1.200000e+01 3.759721e+06 2.326800e+04
  1.440000e+02]
 ...
 [1.000000e+00 2.020000e+03 8.000000e+00 4.080400e+06 1.616000e+04
  6.400000e+01]
 [1.000000e+00 2.020000e+03 9.000000e+00 4.080400e+06 1.818000e+04
  8.100000e+01]
 [1.000000e+00 2.020000e+03 1.000000e+01 4.080400e+06 2.020000e+04
  1.000000e+02]]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[[1.000000e+00 2.021000e+03 1.000000e+00 4.084441e+06 2.021000e+03
  1.000000e+00]
 [1.000000e+00 2.021000e+03 2.000000e+00 4.084441e+06 4.042000e+03
  4.000000e+00]
 [1.000000e+00 2.021000e+03 3.000000e+00 4.084441e+06 6.063000e+03
  9.000000e+00]
 [1.000000e+00 2.021000e+03 4.000000e+00 4.084441e+06 8.084000e+03
  1.600000e+01]
 [1.000000e+00 2.021000e+03 5.000000e+00 4.084441e+06 1.010500e+04
  2.500000e+01]
 [1.000000e+00 2.021000e+03 6.000000e+00 4.084441e+06 1.212600e+04
  3.600000e+01]
 [1.000000e+00 2.021000e+03 7.000000e+00 4.084441e+06 1.414700e+04
  4.900000e+01]
 [1.000000e+00 2.021000e+03 8.000000e+00 4.084441e+06 1.616800e+04
  6.400000e+01]
 [1.000000e+00 2.021000e+03 9.000000e+00 4.084441e+06 1.818900e+04
  8.100000e+01]
 [1.000000e+00 2.021000e+03 1.000000e+01 4.084441e+06 2.021000e+04
  1.000000e+02]
 [1.000000e+00 2.021000e+03 1.100000e+01 4.084441e+06 2.223100e+04
  1.210000e+02]
 [1.000000e+00 2.021000e+03 1.200000e+01 4.084441e+06 2.425200e+04
  1.440000e+02]]


In [ ]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import cross_val_score
C = [1, 5, 10, 25, 100]
m =[]
s = []
for c in C:
    lasso = Lasso(alpha=1/c)
    k = KFold(n_splits=5)
    t = []
    for tr, te in kf.split(X):
        lasso.fit(X[tr], y[tr])   
        yp = lasso.predict(X[te])
        t.append(y[te], ypred)
    t = np.array(t)
    m.append(t.mean())
    s.append(t.std())
plt.errorbar(C, m, yerr=s, linewidth=3)
plt.xlabel('C values')
plt.ylabel('Mean Squared Error')
plt.show()
    
    
   
    